# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1. Read tic_tac_toe.csv into a dataframe
df = pd.read_csv('tic-tac-toe.csv') 
# 2. Inspect the dataset
print("Head of the dataset:")
print(df.head())
print("\nInfo about the dataset:")
print(df.info())

# 3. Convert the categorical values ('x', 'o', 'b') to numeric in all columns
#    'x' -> 1, 'o' -> -1, 'b' -> 0 (as one possibility)
mapping = {'x': 1, 'o': -1, 'b': 0}
for col in df.columns[:-1]:  # Exclude the last column, which is WIN
    df[col] = df[col].map(mapping)

print(df.columns)
print(df.head())

# 4. Separate the inputs (X) and outputs (y)
X = df.drop('class', axis=1)
y = df['class']

# If y is still non-numeric (like T/F or 1/0), ensure it’s binary numeric:
# e.g. T -> 1, F -> 0
y = y.map({True: 1, False: 0})

# 5. Normalize the input data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

print("\nSample of scaled X:")
print(X_scaled[:5])
print("\nSample of y:")
print(y[:5])

Head of the dataset:
  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True

Info about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None
Index(['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR', 'class'], dtype='object')
   TL  TM  TR  ML  MM  MR  BL  BM  BR  class
0   1   1   1 

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [8]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 1. Split the dataset into training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, 
    y, 
    test_size=0.2,   # 20% for test
    random_state=42  # for reproducible splits
)

# 2. Define the model
model = Sequential()
# Input dimension is 9 (the 9 board positions)
model.add(Dense(16, input_dim=9, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 3. Compile the model
#    Binary classification problem → "binary_crossentropy"
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 4. Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,             # Adjust as needed
    batch_size=16,         # Adjust as needed
    verbose=1
)

# 5. Evaluate the trained model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# 6. Save your model for later use (optional)
model.save('tic_tac_toe_model.h5')
print("\nModel saved as tic_tac_toe_model.h5")


c:\Users\pbele\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.4470 - loss: 0.7241 - val_accuracy: 0.4896 - val_loss: 0.6920
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5508 - loss: 0.6778 - val_accuracy: 0.6406 - val_loss: 0.6630
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6363 - loss: 0.6520 - val_accuracy: 0.6510 - val_loss: 0.6374
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6275 - loss: 0.6253 - val_accuracy: 0.6562 - val_loss: 0.6193
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6720 - loss: 0.5856 - val_accuracy: 0.6667 - val_loss: 0.6004
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7243 - loss: 0.5577 - val_accuracy: 0.6823 - val_loss: 0.5850
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7275 - loss: 0.5601 - val_accuracy: 0.7083 - val_loss: 0.5717
Epoch 8/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7248 - loss: 0.5540 - val_accuracy: 0.7188 - val_loss


Test Loss: 0.4792
Test Accuracy: 0.7552

Model saved as tic_tac_toe_model.h5


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [9]:

import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('tic_tac_toe_model.h5')
print("\nModel loaded successfully!")

# Make predictions on a few random rows from the test set
random_indices = np.random.choice(len(X_test), 5, replace=False)
X_sample = X_test[random_indices]
y_sample_true = y_test.iloc[random_indices]

predictions = loaded_model.predict(X_sample)
predicted_classes = [1 if p >= 0.5 else 0 for p in predictions]  # Threshold at 0.5

print("\nSample Predictions vs. Actual:")
for i in range(len(X_sample)):
    print(f"Predicted: {predicted_classes[i]}, Actual: {y_sample_true.iloc[i]}")



Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

Sample Predictions vs. Actual:
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 0, Actual: 0
Predicted: 1, Actual: 0


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# A deeper/wider model, different hyperparameters, more epochs, etc.

model_improved = Sequential()
model_improved.add(Dense(32, input_dim=9, activation='relu'))
model_improved.add(Dense(16, activation='relu'))
model_improved.add(Dense(8, activation='relu'))
model_improved.add(Dense(1, activation='sigmoid'))

# Compile with a different learning rate
optimizer_improved = Adam(learning_rate=0.0005)
model_improved.compile(optimizer=optimizer_improved, loss='binary_crossentropy', metrics=['accuracy'])

history_improved = model_improved.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=40,     # more epochs
    batch_size=16,
    verbose=1
)


loss_improved, accuracy_improved = model_improved.evaluate(X_test, y_test, verbose=0)
print("\nImproved Model Test Loss:", f"{loss_improved:.4f}")
print("Improved Model Test Accuracy:", f"{accuracy_improved:.4f}")


Epoch 1/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3983 - loss: 0.7508 - val_accuracy: 0.5938 - val_loss: 0.6777
Epoch 2/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6622 - loss: 0.6618 - val_accuracy: 0.6562 - val_loss: 0.6252
Epoch 3/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6671 - loss: 0.6145 - val_accuracy: 0.6562 - val_loss: 0.5993
Epoch 4/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6861 - loss: 0.5801 - val_accuracy: 0.6823 - val_loss: 0.5800
Epoch 5/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7009 - loss: 0.5605 - val_accuracy: 0.7083 - val_loss: 0.5647
Epoch 6/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7233 - loss: 0.5573 - val_accuracy: 0.7344 - val_loss: 0.5519
Epoch 7/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7109 - loss: 0.5506 - val_accuracy: 0.7344 - val_loss: 0.5413
Epoch 8/40
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7618 - loss: 0.5149 - val_accuracy: 0.7500 - val_loss:

**Which approach(es) did you find helpful to improve your model performance?**

Adding More Layers: Sometimes a deeper network can capture more complex patterns.
Tuning Hyperparameters: Adjusting the learning rate or the number of neurons in each layer can improve performance.
Training for More Epochs: Can help the model converge to better weights, provided we have enough data and we watch out for overfitting.